# Oxytocin analysis code

In [ ]:
import utility_functions # loading and processing code

# matplotlib stuff
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon # errorbars and standard deviations
%matplotlib qt

# data analysis standards
import numpy as np
import pandas as pd
from scipy import signal
import os, re

# csv writing for figure data etc
import csv


ModuleNotFoundError: No module named 'kilosort'

# Data import and cleaning

Bring in the data, do some basic cleaning before sending it to Kilosort etc

In [ ]:
# recording directory
# assume all subdirectories are from the same session and to be concatenated
recording_dir = ''



